# Data ADS

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

import matplotlib.pyplot as plt

In [2]:
# 1. Đọc danh sách ASIN từ file txt
with open('ASIN.txt', 'r') as f:
    asin_list = [line.strip() for line in f.readlines()]

# 2. Tạo list chứa các file .xlsx trong thư mục Data
folder_path = 'Data_ADS'
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# 3. Đọc từng file, xử lý rồi append vào list
all_data = []

for file in files:
    # Trích xuất ngày từ tên file
    date_str = file.split('_')[1].replace('.xlsx', '')  # "20250713"
    date_obj = datetime.strptime(date_str, "%Y%m%d")     # datetime object
    df = pd.read_excel(os.path.join(folder_path, file))
    
    # Thêm cột date định dạng chuẩn
    df['date'] = date_obj
    
    # Tạo cột ASIN từ 10 ký tự đầu tiên của cột Portfolio
    df['ASIN'] = df['Portfolio'].astype(str).str[:10]
    
    all_data.append(df)

# 4. Gộp tất cả thành một DataFrame
merged_df = pd.concat(all_data, ignore_index=True)

# 5. Lọc chỉ giữ lại các dòng có ASIN nằm trong danh sách
filtered_df = merged_df[merged_df['ASIN'].isin(asin_list)]

#6. Di chuuyển cột ASIN lên đầu, sau Campaign type	
filtered_df = filtered_df[['Campaign type', 'ASIN'] + [col for col in filtered_df.columns if col not in ['Campaign type', 'ASIN']]]

# ✅ Kết quả cuối cùng
filtered_df.head()

,Campaign type,ASIN,Campaign,Status,Country,Profile,Portfolio,Target type,Daily Budget,Current Budget,...,ACOS,ROAS,CPA,Sales Same SKU,Sales Other SKU,Orders Same SKU,Orders Other SKU,Units Same SKU,Units Other SKU,date
23,sponsoredProducts,B08R8R2LQF,B08R8R2LQF_20oz_May the forties black_40th bir...,Out Of Budget,US,NewEleven,B08R8R2LQF_TUMBLER 20_MAY THE FORTIES BE WITH ...,manual,$25.00,$0.00,...,1.0379,0.96,23.85,22.98,0.00,1,--,1,--,2025-07-27
48,sponsoredProducts,B0F2T4JDF7,B0F2T4JDF7_Card_Youre awesome mom_birthday car...,Out Of Budget,US,NewEleven,B0F2T4JDF7_CARD_YOU'RE AN AWESOME MOM_TRINH,manual,$10.00,$0.00,...,0.4805,2.08,4.80,39.96,0.00,4,--,4,--,2025-07-27
50,sponsoredProducts,B092HNBJT6,B092HNBJT6_20oz_Coworker black_auto 15h40 (max...,Out Of Budget,US,NewEleven,B092HNBJT6_TUMBLER 20_COWORKER BLACK_TRINH,auto,$15.00,$0.00,...,0.473,2.11,9.45,0.00,39.96,0,2,0,2,2025-07-27
52,sponsoredProducts,B0DH86X93V,B0DH86X93V_20oz_vintage 1985 weird navy_40th b...,Delivering,US,NewEleven,B0DH86X93V_TUMBLER 20_VINTAGE 1985 WEIRD BEING...,manual,$24.00,$0.00,...,0.4381,2.28,9.41,19.98,22.98,1,1,1,1,2025-07-27
70,sponsoredProducts,B0D9L8P1XL,B0D9L8P1XL_Candle_I have everyone_best friend ...,Out Of Budget,US,NewEleven,B0D9L8P1XL_CANDLE_I HATE EVERYONE EXCEPT US GL...,manual,$15.00,$0.00,...,0.4235,2.36,7.62,17.98,17.98,1,1,1,1,2025-07-27


In [3]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/ppcintern/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1n2Ug3joJwsuuSsrx6T92Zi_dihht7KzIebRTGpKnehc"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("DATA_ADS_TH_CAMP")

set_with_dataframe(sheet1, filtered_df)

# Total GMV

In [12]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

import matplotlib.pyplot as plt

In [13]:
# 1. Đọc danh sách ASIN từ file txt
with open('ASIN.txt', 'r') as f:
    asin_list = [line.strip() for line in f.readlines()]

# 2. Tạo list chứa các file .xlsx trong thư mục Data
folder_path = 'Data_Total'
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 3. Đọc từng file, xử lý rồi append vào list
all_data = []

for file in files:
    # Trích xuất ngày từ tên file
    date_str = file.split('_')[1].replace('.csv', '')  # "20250713"
    date_obj = datetime.strptime(date_str, "%Y%m%d")     # datetime object
    df = pd.read_csv(os.path.join(folder_path, file))
    
    # Thêm cột date định dạng chuẩn
    df['date'] = date_obj
    
    all_data.append(df)

# 4. Gộp tất cả thành một DataFrame
merged_df = pd.concat(all_data, ignore_index=True)



# ✅ Kết quả cuối cùng
merged_df.head()

,(Parent) ASIN,(Child) ASIN,Title,SKU,Sessions - Total,Sessions - Total - B2B,Session Percentage - Total,Session Percentage - Total - B2B,Page Views - Total,Page Views - Total - B2B,...,Featured Offer (Buy Box) Percentage - B2B,Units Ordered,Units Ordered - B2B,Unit Session Percentage,Unit Session Percentage - B2B,Ordered Product Sales,Ordered Product Sales - B2B,Total Order Items,Total Order Items - B2B,date
0,B0F18CMJ3L,B0F18CMJ3L,NewEleven Retirement Gifts For Men 2025 - Cowo...,CU-4EWE-9UH1,167,1,2.82%,0.96%,239,2,...,100.00%,39,1,23.35%,100.00%,$896.22,$22.98,39,1,2025-07-22
1,B0FD9QBGZH,B0DH7YGD9Q,NewEleven 50th Birthday Gifts For Men Women - ...,EC-NYJH-0YZA,174,4,2.94%,3.85%,237,4,...,100.00%,35,1,20.11%,25.00%,$699.30,$19.98,35,1,2025-07-22
2,B0F8QTMBRV,B0DGGDX9P5,"NewEleven 40th Birthday Gifts For Women, Her -...",EI-6RN9-0Z0A,139,4,2.35%,3.85%,208,5,...,100.00%,30,0,21.58%,0.00%,$509.40,$0.00,30,0,2025-07-22
3,B0F7H2RCV2,B0DH88L44J,NewEleven Retirement Gift For Woman 2025 - Coo...,3E-J0TV-6W9S,183,3,3.09%,2.88%,246,3,...,100.00%,28,0,15.30%,0.00%,$643.44,$0.00,28,0,2025-07-22
4,B0FG7QMV1Y,B0D7HMXCSP,"NewEleven Retirement Gifts, Farewell Gift, Goo...",UB-TA3P-4YBQ,108,1,1.83%,0.96%,156,1,...,100.00%,18,0,16.67%,0.00%,$287.64,$0.00,18,0,2025-07-22


In [14]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/ppcintern/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1n2Ug3joJwsuuSsrx6T92Zi_dihht7KzIebRTGpKnehc"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("DATA_BR_ASIN")

set_with_dataframe(sheet1, merged_df)